In [4]:
import tkinter as tk
from tkinter import messagebox, ttk
import random
import string
import sqlite3
import hashlib
from cryptography.fernet import Fernet
import os

# ---------- SECURITY SETUP ----------
KEY_FILE = "secret.key"

# Load existing key or create a new one
def load_or_create_key():
    if os.path.exists(KEY_FILE):
        with open(KEY_FILE, "rb") as key_file:
            return key_file.read()
    else:
        key = Fernet.generate_key()
        with open(KEY_FILE, "wb") as key_file:
            key_file.write(key)
        return key

SECRET_KEY = load_or_create_key()
cipher = Fernet(SECRET_KEY)

# Hash function for login password
def hash_password(password: str) -> str:
    return hashlib.sha256(password.encode()).hexdigest()

# Hardcoded login credentials (hashed password)
VALID_USERNAME = "Manish"
VALID_PASSWORD_HASH = hash_password("M@nish28")  # store hash instead of plain text

# ---------- PASSWORD STRENGTH CHECK ----------
def check_password_strength(password: str) -> int:
    """Return score 1=weak, 2=medium, 3=strong"""
    score = 0
    if len(password) >= 8:
        score += 1
    if any(c.isdigit() for c in password) and any(c.isalpha() for c in password):
        score += 1
    if any(c in string.punctuation for c in password):
        score += 1
    return min(score, 3)

# ---------- PASSWORD POLICY ----------
def validate_password_policy(password: str) -> bool:
    if len(password) < 8:
        messagebox.showwarning("Weak Password", "Password must be at least 8 characters long")
        return False
    if not any(c.isupper() for c in password):
        messagebox.showwarning("Weak Password", "Password must include an uppercase letter")
        return False
    if not any(c.islower() for c in password):
        messagebox.showwarning("Weak Password", "Password must include a lowercase letter")
        return False
    if not any(c.isdigit() for c in password):
        messagebox.showwarning("Weak Password", "Password must include a digit")
        return False
    if not any(c in string.punctuation for c in password):
        messagebox.showwarning("Weak Password", "Password must include a special character")
        return False
    return True

# ---------- MAIN PASSWORD GENERATOR APP ----------
def open_password_generator():
    pw = tk.Tk()
    pw.title("Password Generator")
    pw.geometry("600x500")
    pw.configure(bg="orange")

    tk.Label(pw, text=":PASSWORD GENERATOR:", font=('Helvetica', 18, 'bold', 'underline'), bg="orange", fg="blue").pack(pady=10)

    tk.Label(pw, text="Enter User Name:", font=('Helvetica', 12, 'bold'), bg="orange").place(x=100, y=80)
    username_entry = tk.Entry(pw, width=30)
    username_entry.place(x=300, y=80)

    tk.Label(pw, text="Enter Password Length:", font=('Helvetica', 12, 'bold'), bg="orange").place(x=100, y=120)
    length_entry = tk.Entry(pw, width=30)
    length_entry.place(x=300, y=120)

    tk.Label(pw, text="Generated Password:", font=('Helvetica', 12, 'bold'), bg="orange").place(x=100, y=160)
    password_display = tk.Entry(pw, width=30)
    password_display.place(x=300, y=160)

    # Password strength meter
    strength_label = tk.Label(pw, text="Strength: ", font=('Helvetica', 12, 'bold'), bg="orange")
    strength_label.place(x=100, y=200)

    strength_bar = ttk.Progressbar(pw, length=200, mode="determinate", maximum=3)
    strength_bar.place(x=300, y=200)

    # Create SQLite database and table with UNIQUE constraint
    conn = sqlite3.connect("passwords.db")
    cursor = conn.cursor()
    cursor.execute('''CREATE TABLE IF NOT EXISTS passwords (
                        username TEXT NOT NULL UNIQUE,
                        password BLOB NOT NULL)''')  # UNIQUE added
    conn.commit()

    def update_strength_meter(event=None):
        password = password_display.get()
        score = check_password_strength(password)

        strength_bar["value"] = score
        if score == 1:
            strength_label.config(text="Strength: Weak", fg="red")
        elif score == 2:
            strength_label.config(text="Strength: Medium", fg="orange")
        elif score == 3:
            strength_label.config(text="Strength: Strong", fg="green")
        else:
            strength_label.config(text="Strength: ", fg="black")

    def generate_password():
        username = username_entry.get()
        try:
            length = int(length_entry.get())
        except ValueError:
            messagebox.showerror("Invalid", "Enter a valid number for length")
            return

        if username == "" or length < 8:
            messagebox.showwarning("Invalid", "Username is empty or length < 8")
            return

        characters = string.ascii_letters + string.digits + string.punctuation
        password = ''.join(random.sample(characters, length))
        password_display.delete(0, tk.END)
        password_display.insert(0, password)
        update_strength_meter()

    def accept_password():
        username = username_entry.get()
        password = password_display.get()
        if username == "" or password == "":
            messagebox.showwarning("Warning", "Fields cannot be empty")
            return

        # ✅ Enforce policy before saving
        if not validate_password_policy(password):
            return

        # Encrypt password before saving
        encrypted_password = cipher.encrypt(password.encode())
        try:
            cursor.execute("INSERT INTO passwords VALUES (?, ?)", (username, encrypted_password))
            conn.commit()
            messagebox.showinfo("Saved", "Username and password saved securely.")
        except sqlite3.IntegrityError:
            messagebox.showerror("Duplicate Username", f"Username '{username}' already exists! Please use a different one.")

    def reset_fields_and_data():
        username_entry.delete(0, tk.END)
        length_entry.delete(0, tk.END)
        password_display.delete(0, tk.END)
        strength_bar["value"] = 0
        strength_label.config(text="Strength: ", fg="black")
        cursor.execute("DELETE FROM passwords")  # Clear database
        conn.commit()
        messagebox.showinfo("Reset", "Fields cleared and all saved passwords deleted.")

    def view_saved_passwords():
        cursor.execute("SELECT * FROM passwords")
        records = cursor.fetchall()

        if not records:
            messagebox.showinfo("No Records", "No saved passwords found.")
            return

        view_win = tk.Toplevel(pw)
        view_win.title("Saved Passwords")
        view_win.geometry("400x300")
        view_win.configure(bg="lightyellow")

        tk.Label(view_win, text="Saved Passwords:", font=('Helvetica', 12, 'bold'), bg="lightyellow").pack(pady=10)

        for username, encrypted_password in records:
            if isinstance(encrypted_password, str):
                encrypted_password = encrypted_password.encode()
            try:
                decrypted_pass = cipher.decrypt(encrypted_password).decode()
            except Exception:
                decrypted_pass = "<Unable to decrypt - key mismatch>"
            tk.Label(view_win, text=f"Username: {username} | Password: {decrypted_pass}", bg="lightyellow", anchor="w").pack()

    # Buttons
    tk.Button(pw, text="GENERATE PASSWORD", font=('Helvetica', 12, 'bold'),
              command=generate_password, bg="yellowgreen").place(x=200, y=250)

    tk.Button(pw, text="ACCEPT", font=('Helvetica', 12, 'bold'),
              command=accept_password, bg="lightgreen").place(x=250, y=300)

    tk.Button(pw, text="RESET", font=('Helvetica', 12, 'bold'),
              command=reset_fields_and_data, bg="lightgreen").place(x=250, y=350)

    tk.Button(pw, text="VIEW SAVED PASSWORDS", font=('Helvetica', 12, 'bold'),
              command=view_saved_passwords, bg="skyblue").place(x=200, y=400)

    password_display.bind("<KeyRelease>", update_strength_meter)

# ---------- LOGIN SCREEN ----------
def login_screen():
    root = tk.Tk()
    root.title("Login")
    root.geometry("600x400")
    root.configure(bg="lightblue")

    tk.Label(root, text="LOGIN", font=('Helvetica', 18, 'bold')).pack(pady=10)

    tk.Label(root, text="Username:", font=('Helvetica', 12)).place(x=150, y=120)
    username_entry = tk.Entry(root, width=30)
    username_entry.place(x=270, y=120)

    tk.Label(root, text="Password:", font=('Helvetica', 12)).place(x=150, y=170)
    password_entry = tk.Entry(root, width=30, show="*")
    password_entry.place(x=270, y=170)

    def verify_login():
        entered_username = username_entry.get()
        entered_password = password_entry.get()

        if entered_username == VALID_USERNAME and hash_password(entered_password) == VALID_PASSWORD_HASH:
            messagebox.showinfo("Success", "Login Successful")
            root.destroy()
            open_password_generator()
        else:
            messagebox.showerror("Error", "Incorrect Username or Password")

    tk.Button(root, text="Login", font=('Helvetica', 12, 'bold'), command=verify_login, bg="green").place(x=250, y=230)

    root.mainloop()

# ---------- RUN LOGIN ----------
login_screen()


In [2]:
import sqlite3

# open db
conn = sqlite3.connect("passwords.db")
cursor = conn.cursor()

# show all saved data
cursor.execute("SELECT * FROM passwords")
rows = cursor.fetchall()

for row in rows:
    print(row)

conn.close()

('Annu Sharma', b'gAAAAABo0kA86TRC7CZusUBcFeHN4PmLJZWn11ZK0AZn_f-wcte8jSu9Q5bnb1LutmCXuO0TXniv7CnkM0V-c9QJo1L5Bd70_w==')
('Amit', b'gAAAAABo0kBbqFOcbxmY9WuGOI6Hkp7x_F7q8YfMjp1FuvB14F9JljsgCq78IP7LELSDKz6d2gZ8kUNhNFPILNFjrT6C9frOhQ==')


In [3]:
import sqlite3
from cryptography.fernet import Fernet

# Load secret key
with open("secret.key", "rb") as f:
    key = f.read()

fernet = Fernet(key)

# Open DB
conn = sqlite3.connect("passwords.db")
cursor = conn.cursor()

cursor.execute("SELECT * FROM passwords")
rows = cursor.fetchall()

# Decrypt and print
for row in rows:
    username = row[0]
    encrypted_password = row[1]
    decrypted_password = fernet.decrypt(encrypted_password).decode()
    print(f"Username: {username}, Password: {decrypted_password}")

conn.close()

Username: Annu Sharma, Password: |e1@Q.Ej>
Username: Amit, Password: !Oofzc3$MG
